## Librerias

import pandas as pd
from pathlib import Path
import numpy as np
import os
import datetime as dt
import warnings
import re, unicodedata


In [283]:
import pandas as pd
from pathlib import Path
import numpy as np
import os
import datetime as dt
import warnings
import re, unicodedata
import warnings

warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl")

In [284]:
# Fijar directorio
DATA_DIR = Path(rf"C:\Users\PC\Desktop\Proyectos\Proyectos_Py\1.Analisis Vivienda\Analisis_Vivienda\data\raw")
DATA_OUT = Path(rf"C:\Users\PC\Desktop\Proyectos\Proyectos_Py\1.Analisis Vivienda\Analisis_Vivienda\data\processed")

In [285]:
# Lista todos los archivos .csv
csv_files = sorted(DATA_DIR.glob("*.csv"))
print(f"Se listan {len(csv_files)} archivos a procesar:")
for f in csv_files:
    print("-", f.name)

## Lectura de datos
data_raw = []
cols_by_file = {}
for file in csv_files:
    df = pd.read_csv(file, sep="|")
    df["file_origin"] = file.name
    data_raw.append(df)
    cols_by_file[file.name] = set(df.columns)
    
print("\nproceso cargado!")

print("\nUniendo datos!")
raw_all = pd.concat(data_raw, ignore_index=True)
print("Filas totales:", len(raw_all))
print("Columnas:", list(raw_all.columns))

Se listan 35 archivos a procesar:
- data_alquiler_departamentos_adondevivir_Lima Centro.csv
- data_alquiler_departamentos_adondevivir_Lima Este.csv
- data_alquiler_departamentos_adondevivir_Lima Moderna.csv
- data_alquiler_departamentos_adondevivir_Lima Norte.csv
- data_alquiler_departamentos_adondevivir_Lima Sur.csv
- data_alquiler_departamentos_adondevivir_Lima Top.csv
- data_alquiler_departamentos_urbania_Lima Centro.csv
- data_alquiler_departamentos_urbania_Lima Este.csv
- data_alquiler_departamentos_urbania_Lima Moderna.csv
- data_alquiler_departamentos_urbania_Lima Norte.csv
- data_alquiler_departamentos_urbania_Lima Sur.csv
- data_alquiler_departamentos_urbania_Lima Top.csv
- data_venta_casas_adondevivir_Lima Centro.csv
- data_venta_casas_adondevivir_Lima Este.csv
- data_venta_casas_adondevivir_Lima Moderna.csv
- data_venta_casas_adondevivir_Lima Norte.csv
- data_venta_casas_adondevivir_Lima Sur.csv
- data_venta_casas_adondevivir_Lima Top.csv
- data_venta_casas_urbania_Lima Cent

In [286]:
print("\nLimpieza de datos!")

## Limpieza de datos - duplicados Otros. 

dups = raw_all["enlace"].duplicated(keep=False)
print("Registros duplicados por enlace:", dups.sum())

# ¿Cuántos enlaces únicos?
print("Enlaces únicos:", raw_all["enlace"].nunique())

## removeremos duplicados.
raw_all_nd = raw_all.drop_duplicates(subset=["enlace"], keep="first") 
print("se tiene solo enlaces unicos:", raw_all_nd["enlace"].nunique())


Limpieza de datos!
Registros duplicados por enlace: 159
Enlaces únicos: 49767
se tiene solo enlaces unicos: 49767


In [287]:
valores = {
    "Brena": "Breña",
    "Jesus Maria": "Jesús María",
    "San Martin De Porres": "San Martín de Porres",
    "Lurin": "Lurín",
    "Ancon": "Ancón",
    "Santa Maria Del Mar": "Santa María del Mar",
    "Villa Maria Del Triunfo": "Villa María del Triunfo",
}

s = (raw_all_nd["distrito"]
        .astype(str)
        .str.strip()
        .str.replace(r"\s+", " ", regex=True))

raw_all_nd["distrito"] = s.replace(valores)   # crea la nueva columna

raw_all_nd["distrito_norm"] = raw_all_nd["distrito"].str.lower().str.strip()
raw_all_nd["direccion_norm"] = raw_all_nd["direccion"].str.lower().str.strip()

raw_all_nd["distrito_en_direccion"] = raw_all_nd.apply(
    lambda row: row["distrito_norm"] in row["direccion_norm"],
    axis=1
)

C:\Users\PC\AppData\Local\Temp\ipykernel_24912\3101019241.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_all_nd["distrito"] = s.replace(valores)   # crea la nueva columna
C:\Users\PC\AppData\Local\Temp\ipykernel_24912\3101019241.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_all_nd["distrito_norm"] = raw_all_nd["distrito"].str.lower().str.strip()
C:\Users\PC\AppData\Local\Temp\ipykernel_24912\3101019241.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

In [288]:
cols = ["distrito", "direccion", 
        "distrito_norm", "direccion_norm", "distrito_en_direccion"]
raw_all_nd[raw_all_nd["distrito_en_direccion"]==False][cols]

,distrito,direccion,distrito_norm,direccion_norm,distrito_en_direccion
60,Breña,"Mariano Carranza al 400\nSanta Beatriz, Lima C...",breña,"mariano carranza al 400\nsanta beatriz, lima c...",False
61,Breña,"faustino sanchez carrion 397\nJesús María, Lima",breña,"faustino sanchez carrion 397\njesús maría, lima",False
62,Breña,"Jirón Estados Unidos 801\nJesús María, Lima",breña,"jirón estados unidos 801\njesús maría, lima",False
63,Breña,"Jirón Diego de Almagro\nJesús María, Lima",breña,"jirón diego de almagro\njesús maría, lima",False
64,Breña,"Calle Arnaldo Marquez 1440\nJesús María, Lima",breña,"calle arnaldo marquez 1440\njesús maría, lima",False
...,...,...,...,...,...
39209,Villa Maria del Triunfo,Jr. José Olaya Mz. D6 Lt. 12\nSanta Rosa De Be...,villa maria del triunfo,jr. josé olaya mz. d6 lt. 12\nsanta rosa de be...,False
39210,Villa Maria del Triunfo,Villa María Del Triunfo\nVilla María del Triun...,villa maria del triunfo,villa maría del triunfo\nvilla maría del triun...,False
39211,Villa Maria del Triunfo,"Castilla 245\nVilla María del Triunfo, Lima",villa maria del triunfo,"castilla 245\nvilla maría del triunfo, lima",False
39212,Villa Maria del Triunfo,"Castilla 245\nVilla María del Triunfo, Lima",villa maria del triunfo,"castilla 245\nvilla maría del triunfo, lima",False


In [289]:
canon = {
    # canónico : [alias que podrían aparecer en la dirección]
    "Ate Vitarte": ["ate vitarte", "ate"],
    "Breña": ["breña", "brena"],
    "Carabayllo": ["carabayllo"],
    "Chaclacayo": ["chaclacayo"],
    "El Agustino": ["el agustino", "agustino"],
    "Independencia": ["independencia"],
    "Jesús María": ["jesus maria", "jesús maria", "jesus maría"], ## Jesús María
    "Chosica Lurigancho": [ "chosica"],
    "Lurín": ["lurin", "lurín"],
    "Pachacamac": ["pachacamac", "pachacámac"],
    "Pucusana": ["pucusana"],
    "Puente Piedra": ["puente piedra"],
    "Punta Negra": ["punta negra"],
    "Rimac": ["rimac", "rímac"],
    "San Juan de Miraflores": ["san juan de miraflores", "sjm"],
    "San Luis": ["san luis"],
    "San Martin de Porres": ["san martin de porres", "san martín de porres", "smp"],
    "Villa Maria del Triunfo": ["villa maria del triunfo", "villa maría del triunfo", "vmt"],
    "Ancón": ["ancon", "ancón"],
    "Lince": ["lince"],
    "Santa Rosa": ["santa rosa"],
    "Barranco": ["barranco"],
    "Magdalena": ["magdalena del mar", "magdalena"],
    "Punta Hermosa": ["punta hermosa"],
    "Santiago de Surco": ["santiago de surco", "surco"],
    "San Juan de Lurigancho": ["san juan de lurigancho"],
    "Lima Cercado" : ["cercado", "cercado de lima", "lima cercado"],
    "Los Olivos" : ["los olivos"],
    "Comas" : ["comas"],
    "San Bartolo" : ["san bartolo"], 
    'Villa El Salvador' : ["villa el salvador"],
    "Santa Maria del Mar": ["santa maria del mar"],
    "San Isidro" : ["san isidro"],
    "La Molina" : ["la molina"],
    "Chorrillos" : ["chorrillos"],
    "San Borja" : ["san borja"],
    "Miraflores" : ["miraflores"],
    "Surquillo" : ["surquillo"],
    "Cieneguilla" : ["cieneguilla"],
    "Ventanilla" : ["ventanilla"],
    "La Victoria" : ["la victoria"],
    'Pueblo Libre' : ["pueblo libre"], 
    'San Miguel' : ["san miguel"], 
    'Santa Anita' : ["santa anita"], 
    'Ventanilla' : ["ventanilla"], 
}

def strip_accents(s):
    if not isinstance(s, str):
        return s
    return ''.join(ch for ch in unicodedata.normalize("NFD", s) if unicodedata.category(ch) != "Mn")

raw_all_nd["dir_norm"]  = raw_all_nd["direccion"].astype(str).str.lower().str.replace(r"\s+", " ", regex=True).map(strip_accents)
raw_all_nd["dist_norm"] = raw_all_nd["distrito"].astype(str).str.lower().str.replace(r"\s+", " ", regex=True).map(strip_accents)

# precompila patrones regex alias
patterns = []
for canon_name, aliases in canon.items():
    for alias in aliases:
        pat = re.compile(rf"\b{re.escape(strip_accents(alias))}\b")
        patterns.append((canon_name, pat))

def candidates_from_address(addr_norm: str):
    if not isinstance(addr_norm, str):
        return []
    hits = []
    for canon_name, pat in patterns:
        if pat.search(addr_norm):
            hits.append(canon_name)
    # quitar duplicados preservando orden
    return  list(dict.fromkeys(hits))

raw_all_nd["dist_candidates"] = raw_all_nd["dir_norm"].map(candidates_from_address)
raw_all_nd["distrito_candidato"] = raw_all_nd["dist_candidates"].str.get(0) 

C:\Users\PC\AppData\Local\Temp\ipykernel_24912\2080104380.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_all_nd["dir_norm"]  = raw_all_nd["direccion"].astype(str).str.lower().str.replace(r"\s+", " ", regex=True).map(strip_accents)
C:\Users\PC\AppData\Local\Temp\ipykernel_24912\2080104380.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_all_nd["dist_norm"] = raw_all_nd["distrito"].astype(str).str.lower().str.replace(r"\s+", " ", regex=True).map(strip_accents)
C:\Users\PC\AppData\Local\Temp\

In [290]:
# raw_all_nd[(raw_all_nd["distrito_en_direccion"]==False) & (raw_all_nd["distrito_candidato"]).isnull()][cols + ["distrito_candidato", "enlace", "operacion"]]

In [291]:
raw_all_nd2 = raw_all_nd[(raw_all_nd["distrito_candidato"].notnull()) & (raw_all_nd["distrito_candidato"] != "Ventanilla") ].copy() ## no van distritos de callao ni distritos candidatos qu eno pertenezcan a lima

In [292]:
cols_obj =['fuente', 'inmueble', 'operacion', 'distrito_candidato' , 'zona', 'direccion', 'precio',
            'soles', 'dolares', 'variacion', 'mantenimiento', 'caracteristica',
            'area', 'dormitorio', 'baños', 'estacionamientos', 'detalle', 'enlace',
            'fecha', 'file_origin']

data_vivienda = raw_all_nd2[cols_obj].copy()

def normalizar(texto: str) -> str:
    # 1. Pasar a minúsculas
    texto = texto.lower()
    # 2. Quitar acentos
    texto = ''.join(
        c for c in unicodedata.normalize('NFD', texto)
        if unicodedata.category(c) != 'Mn'
    )
    # 3. Quitar dobles espacios y limpiar
    texto = re.sub(r'\s+', ' ', texto).strip()
    return texto


data_vivienda["distrito_oficial"] = data_vivienda["distrito_candidato"] ## .apply(normalizar)

In [293]:
data_vivienda.head()

,fuente,inmueble,operacion,distrito_candidato,zona,direccion,precio,soles,dolares,variacion,...,caracteristica,area,dormitorio,baños,estacionamientos,detalle,enlace,fecha,file_origin,distrito_oficial
0,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Mariano Carranza al 400\nSanta Beatriz, Lima C...","S/ 2,300 · USD 683",2300.0,683.0,NaN,...,61 m² tot.\n2 dorm.\n2 baños,61,2,2.0,NaN,Departamento de estreno con Balcón y Excelente...,https://www.adondevivir.com/propiedades/clasif...,2025-12-23,data_alquiler_departamentos_adondevivir_Lima C...,Lima Cercado
1,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Avenida Colonial 3046\nLa Colonial, Lima Cercado","S/ 1,700\n5%",1700.0,NaN,5%,...,60 m² tot.\n3 dorm.\n2 baños,60,3,2.0,NaN,Se Alquila Departamento bien iluminado con Bal...,https://www.adondevivir.com/propiedades/clasif...,2025-12-23,data_alquiler_departamentos_adondevivir_Lima C...,Lima Cercado
2,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Parque de la Reserva 736\nLima Cercado, Lima","S/ 2,500",2500.0,NaN,NaN,...,75 m² tot.\n3 dorm.\n2 baños,75,3,2.0,NaN,Contacto: malena peláez // cel: 993*306*450. S...,https://www.adondevivir.com/propiedades/clasif...,2025-12-23,data_alquiler_departamentos_adondevivir_Lima C...,Lima Cercado
3,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Av. Alejandro Bertello Cdra\nPando, Lima Cercado","S/ 1,550",1550.0,NaN,NaN,...,32 m² tot.\n1 dorm.\n1 baño,32,1,NaN,NaN,Alquiler de Departamento en Lima Cercado - Urb...,https://www.adondevivir.com/propiedades/clasif...,2025-12-23,data_alquiler_departamentos_adondevivir_Lima C...,Lima Cercado
4,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Av. Oscar R. Benavides\nLima Cercado, Lima","S/ 1,500",1500.0,NaN,NaN,...,50 m² tot.\n2 dorm.\n2 baños,50,2,2.0,NaN,"Con un área de 50m2, Ubicado en el Piso 14 con...",https://www.adondevivir.com/propiedades/clasif...,2025-12-23,data_alquiler_departamentos_adondevivir_Lima C...,Lima Cercado


In [294]:
data_vivienda[data_vivienda["distrito_candidato"]=="Independencia"].head(50)

,fuente,inmueble,operacion,distrito_candidato,zona,direccion,precio,soles,dolares,variacion,...,caracteristica,area,dormitorio,baños,estacionamientos,detalle,enlace,fecha,file_origin,distrito_oficial
155,adondevivir,departamentos,alquiler,Independencia,Lima Centro,"MegaPlaza\nPanamericana Norte Industrial, Inde...","S/ 1,300 · USD 380",1300.0,380.0,NaN,...,85 m² tot.\n3 dorm.\n2 baños,85,3,2.0,NaN,"Departamento en piso 3, cuenta con 3 habitacio...",https://www.adondevivir.com/propiedades/clasif...,2025-12-23,data_alquiler_departamentos_adondevivir_Lima C...,Independencia
319,adondevivir,departamentos,alquiler,Independencia,Lima Este,"MegaPlaza\nPanamericana Norte Industrial, Inde...","S/ 1,300 · USD 380",1300.0,380.0,NaN,...,85 m² tot.\n3 dorm.\n2 baños,85,3.0,2.0,NaN,"Departamento en piso 3, cuenta con 3 habitacio...",https://www.adondevivir.com/propiedades/clasif...,2025-12-23,data_alquiler_departamentos_adondevivir_Lima E...,Independencia
684,adondevivir,departamentos,alquiler,Independencia,Lima Moderna,"Jr. Independencia 508 San Miguel\nSan Miguel, ...",S/ 880,880.0,NaN,NaN,...,25 m² tot.\n1 dorm.\n1 baño,25,1.0,NaN,NaN,Alquilo ambiente amplio tipo estudio en 5to. p...,https://www.adondevivir.com/propiedades/clasif...,2025-12-31,data_alquiler_departamentos_adondevivir_Lima M...,Independencia
767,adondevivir,departamentos,alquiler,Independencia,Lima Moderna,"jr. Independencia\nSan Miguel, Lima",S/ 900 · USD 240,900.0,240.0,NaN,...,28 m² tot.\n1 dorm.\n1 baño,28,1.0,NaN,NaN,Contáctame Susan Alzamora. 9. 0. 2. 0. 1. 6. 4...,https://www.adondevivir.com/propiedades/clasif...,2025-12-31,data_alquiler_departamentos_adondevivir_Lima M...,Independencia
1133,adondevivir,departamentos,alquiler,Independencia,Lima Norte,"MegaPlaza\nPanamericana Norte Industrial, Inde...","S/ 1,300 · USD 380",1300.0,380.0,NaN,...,85 m² tot.\n3 dorm.\n2 baños,85.0,3.0,2.0,NaN,"Departamento en piso 3, cuenta con 3 habitacio...",https://www.adondevivir.com/propiedades/clasif...,2025-12-23,data_alquiler_departamentos_adondevivir_Lima N...,Independencia
1134,adondevivir,departamentos,alquiler,Independencia,Lima Norte,"Los Melocotones\nIndependencia, Lima",S/ 900 · USD 257,900.0,257.0,NaN,...,77 m² tot.\n2 dorm.\n1 baño,77.0,2.0,NaN,NaN,"Se alquila departamento en 2do piso, de 78m² u...",https://www.adondevivir.com/propiedades/clasif...,2025-12-23,data_alquiler_departamentos_adondevivir_Lima N...,Independencia
1135,adondevivir,departamentos,alquiler,Independencia,Lima Norte,"Los Capulies 121 Independencia\nLas Violetas, ...","S/ 1,800",1800.0,NaN,NaN,...,140 m² tot.\n3 dorm.\n2 baños,140.0,3.0,2.0,NaN,Departamento muy amplio semi amoblado. Todo el...,https://www.adondevivir.com/propiedades/clasif...,2025-12-23,data_alquiler_departamentos_adondevivir_Lima N...,Independencia
1136,adondevivir,departamentos,alquiler,Independencia,Lima Norte,"LAS AMERICAS\nIndependencia, Lima",S/ 899,899.0,NaN,NaN,...,60 m² tot.\n2 dorm.\n1 baño,60.0,2.0,NaN,NaN,*Alquiler de departamento en Independencia! * ...,https://www.adondevivir.com/propiedades/clasif...,2025-12-23,data_alquiler_departamentos_adondevivir_Lima N...,Independencia
1655,adondevivir,departamentos,alquiler,Independencia,Lima Top,"Independencia\nMiraflores, Lima","S/ 3,000",3000.0,NaN,NaN,...,93 m² tot.\n2 dorm.\n2 baños\n1 estac.,93,2,2.0,1.0,Alquiler mensual s/3. 000 Precioso departament...,https://www.adondevivir.com/propiedades/clasif...,2025-12-31,data_alquiler_departamentos_adondevivir_Lima T...,Independencia
1670,adondevivir,departamentos,alquiler,Independencia,Lima Top,"Calle Independencia 4, Piso 4, Miraflores\nMir...","S/ 2,958 · USD 870",2958.0,870.0,NaN,...,80 m² tot.\n2 dorm.\n1 baño\n1 estac.,80,2,NaN,1.0,"Alquilo Lindo Departamento Flat Remodelado, Fu...",https://www.adondevivir.com/propiedades/clasif...,2025-12-31,data_alquiler_departamentos_adondevivir_Lima T...,Independencia


In [295]:

print("\nCreacion de variables!")

cols_analisis = ['fuente','inmueble','operacion',
                'distrito_oficial', 'zona',
                'direccion','precio',
                'soles','dolares','variacion','mantenimiento','caracteristica',
                'area','dormitorio','baños','estacionamientos','detalle','enlace',
                'fecha','file_origin']
data_analisis = data_vivienda[cols_analisis].copy()
data_analisis


Creacion de variables!


,fuente,inmueble,operacion,distrito_oficial,zona,direccion,precio,soles,dolares,variacion,mantenimiento,caracteristica,area,dormitorio,baños,estacionamientos,detalle,enlace,fecha,file_origin
0,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Mariano Carranza al 400\nSanta Beatriz, Lima C...","S/ 2,300 · USD 683",2300.0,683.0,NaN,NaN,61 m² tot.\n2 dorm.\n2 baños,61,2,2.0,NaN,Departamento de estreno con Balcón y Excelente...,https://www.adondevivir.com/propiedades/clasif...,2025-12-23,data_alquiler_departamentos_adondevivir_Lima C...
1,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Avenida Colonial 3046\nLa Colonial, Lima Cercado","S/ 1,700\n5%",1700.0,NaN,5%,160.0,60 m² tot.\n3 dorm.\n2 baños,60,3,2.0,NaN,Se Alquila Departamento bien iluminado con Bal...,https://www.adondevivir.com/propiedades/clasif...,2025-12-23,data_alquiler_departamentos_adondevivir_Lima C...
2,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Parque de la Reserva 736\nLima Cercado, Lima","S/ 2,500",2500.0,NaN,NaN,NaN,75 m² tot.\n3 dorm.\n2 baños,75,3,2.0,NaN,Contacto: malena peláez // cel: 993*306*450. S...,https://www.adondevivir.com/propiedades/clasif...,2025-12-23,data_alquiler_departamentos_adondevivir_Lima C...
3,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Av. Alejandro Bertello Cdra\nPando, Lima Cercado","S/ 1,550",1550.0,NaN,NaN,180.0,32 m² tot.\n1 dorm.\n1 baño,32,1,NaN,NaN,Alquiler de Departamento en Lima Cercado - Urb...,https://www.adondevivir.com/propiedades/clasif...,2025-12-23,data_alquiler_departamentos_adondevivir_Lima C...
4,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Av. Oscar R. Benavides\nLima Cercado, Lima","S/ 1,500",1500.0,NaN,NaN,210.0,50 m² tot.\n2 dorm.\n2 baños,50,2,2.0,NaN,"Con un área de 50m2, Ubicado en el Piso 14 con...",https://www.adondevivir.com/propiedades/clasif...,2025-12-23,data_alquiler_departamentos_adondevivir_Lima C...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49846,urbania,departamentos,venta,Barranco,Lima Top,"Pasaje Bresciani 1\nBarranco, Lima","S/ 6,146,200 · USD 1,580,000",6146200.0,1580000.0,NaN,"4,000",638 m² tot.\n3 dorm.\n3 baños\n7 estac.,638,3,3.0,7.0,"Vendo Lindo departamento frente al Mar, en Pas...",https://urbania.pe/inmueble/clasificado/veclap...,2025-12-22,data_venta_departamentos_urbania_Lima Top.csv
49847,urbania,departamentos,venta,Barranco,Lima Top,"Jirón Tumay 095\nBarranco, Lima","S/ 603,000 · USD 167,500",603000.0,167500.0,NaN,400,62 m² tot.\n2 dorm.\n2 baños,62,2,2.0,NaN,Departamento en venta en BarrancoExcelente ubi...,https://urbania.pe/inmueble/clasificado/veclap...,2025-12-22,data_venta_departamentos_urbania_Lima Top.csv
49848,urbania,departamentos,venta,Barranco,Lima Top,"martinto 150\nBarranco, Lima","S/ 562,800 · USD 168,000",562800.0,168000.0,NaN,200,86 m² tot.\n2 dorm.\n2 baños\n2 estac.,86,2,2.0,2.0,"Lindo dpto. Muy bien distribuido, de 2 Dormito...",https://urbania.pe/inmueble/clasificado/veclap...,2025-12-22,data_venta_departamentos_urbania_Lima Top.csv
49849,urbania,departamentos,venta,Barranco,Lima Top,"Ignacio mariategui\nBarranco, Lima","S/ 351,750 · USD 105,000",351750.0,105000.0,NaN,180,76 m² tot.\n2 dorm.\n1 baño,76,2,NaN,NaN,Hermoso departamento en venta en Barranco en p...,https://urbania.pe/inmueble/clasificado/veclap...,2025-12-22,data_venta_departamentos_urbania_Lima Top.csv


In [298]:
# Normalizaciones básicas
data_analisis["fuente"] = data_analisis["fuente"].astype(str).str.strip().str.lower()
data_analisis["distrito"] = data_analisis["distrito_oficial"].astype(str).str.strip().str.title()
data_analisis["inmueble"] = data_analisis["inmueble"].astype(str).str.strip().str.lower()
data_analisis["operacion"] = data_analisis["operacion"].astype(str).str.strip().str.lower()
data_analisis["precio_pen"] = data_analisis["soles"]
data_analisis["precio_usd"] = np.where(data_analisis["dolares"].isnull(), data_analisis["precio_pen"]/3.40, data_analisis["dolares"])

## solo registros con precios
data_analisis["sin_precio"] = (data_analisis["precio_pen"].isna() & data_analisis["precio_usd"].isna()).astype(int)
data_analisis = data_analisis.query("sin_precio == 0").copy()

# AREA robusta
tmp = data_analisis["area"].astype(str).str.replace(r"\s+", " ", regex=True).str.strip()
mm = tmp.str.split(" a ", n=1, expand=True)
data_analisis["area_min"] = pd.to_numeric(mm[0].str.replace(",", "."), errors="coerce")
data_analisis["area_max"] = pd.to_numeric(mm[1].str.replace(",", "."), errors="coerce")
data_analisis["area_promedio"] = data_analisis[["area_min","area_max"]].mean(axis=1)

data_analisis

,fuente,inmueble,operacion,distrito_oficial,zona,direccion,precio,soles,dolares,variacion,...,enlace,fecha,file_origin,distrito,precio_pen,precio_usd,sin_precio,area_min,area_max,area_promedio
0,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Mariano Carranza al 400\nSanta Beatriz, Lima C...","S/ 2,300 · USD 683",2300.0,683.0,NaN,...,https://www.adondevivir.com/propiedades/clasif...,2025-12-23,data_alquiler_departamentos_adondevivir_Lima C...,Lima Cercado,2300.0,6.830000e+02,0,61.0,NaN,61.0
1,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Avenida Colonial 3046\nLa Colonial, Lima Cercado","S/ 1,700\n5%",1700.0,NaN,5%,...,https://www.adondevivir.com/propiedades/clasif...,2025-12-23,data_alquiler_departamentos_adondevivir_Lima C...,Lima Cercado,1700.0,5.000000e+02,0,60.0,NaN,60.0
2,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Parque de la Reserva 736\nLima Cercado, Lima","S/ 2,500",2500.0,NaN,NaN,...,https://www.adondevivir.com/propiedades/clasif...,2025-12-23,data_alquiler_departamentos_adondevivir_Lima C...,Lima Cercado,2500.0,7.352941e+02,0,75.0,NaN,75.0
3,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Av. Alejandro Bertello Cdra\nPando, Lima Cercado","S/ 1,550",1550.0,NaN,NaN,...,https://www.adondevivir.com/propiedades/clasif...,2025-12-23,data_alquiler_departamentos_adondevivir_Lima C...,Lima Cercado,1550.0,4.558824e+02,0,32.0,NaN,32.0
4,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Av. Oscar R. Benavides\nLima Cercado, Lima","S/ 1,500",1500.0,NaN,NaN,...,https://www.adondevivir.com/propiedades/clasif...,2025-12-23,data_alquiler_departamentos_adondevivir_Lima C...,Lima Cercado,1500.0,4.411765e+02,0,50.0,NaN,50.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49846,urbania,departamentos,venta,Barranco,Lima Top,"Pasaje Bresciani 1\nBarranco, Lima","S/ 6,146,200 · USD 1,580,000",6146200.0,1580000.0,NaN,...,https://urbania.pe/inmueble/clasificado/veclap...,2025-12-22,data_venta_departamentos_urbania_Lima Top.csv,Barranco,6146200.0,1.580000e+06,0,638.0,NaN,638.0
49847,urbania,departamentos,venta,Barranco,Lima Top,"Jirón Tumay 095\nBarranco, Lima","S/ 603,000 · USD 167,500",603000.0,167500.0,NaN,...,https://urbania.pe/inmueble/clasificado/veclap...,2025-12-22,data_venta_departamentos_urbania_Lima Top.csv,Barranco,603000.0,1.675000e+05,0,62.0,NaN,62.0
49848,urbania,departamentos,venta,Barranco,Lima Top,"martinto 150\nBarranco, Lima","S/ 562,800 · USD 168,000",562800.0,168000.0,NaN,...,https://urbania.pe/inmueble/clasificado/veclap...,2025-12-22,data_venta_departamentos_urbania_Lima Top.csv,Barranco,562800.0,1.680000e+05,0,86.0,NaN,86.0
49849,urbania,departamentos,venta,Barranco,Lima Top,"Ignacio mariategui\nBarranco, Lima","S/ 351,750 · USD 105,000",351750.0,105000.0,NaN,...,https://urbania.pe/inmueble/clasificado/veclap...,2025-12-22,data_venta_departamentos_urbania_Lima Top.csv,Barranco,351750.0,1.050000e+05,0,76.0,NaN,76.0


In [299]:
# DORMITORIO robusto
tmpd = data_analisis["dormitorio"].astype(str).str.replace(r"\s+", " ", regex=True).str.strip()
mmd = tmpd.str.split(" a ", n=1, expand=True)
data_analisis["domitorio_min"] = pd.to_numeric(mmd[0], errors="coerce")
data_analisis["domitorio_max"] = pd.to_numeric(mmd[1], errors="coerce")

# Precio por m2 (división segura)
den = data_analisis["area_promedio"].replace(0, np.nan)
data_analisis["precio_m2_pen"] = (data_analisis["precio_pen"] / den).round(2)

# Flags
data_analisis["tiene_mantenimiento"] = data_analisis["mantenimiento"].notna().astype(int)
data_analisis["tiene_estacionamientos"] = data_analisis["estacionamientos"].notna().astype(int)
data_analisis["banios"] = data_analisis["caracteristica"].str.extract(r"(\d+)\s*bañ").astype("Int64")
data_analisis.head()

,fuente,inmueble,operacion,distrito_oficial,zona,direccion,precio,soles,dolares,variacion,...,sin_precio,area_min,area_max,area_promedio,domitorio_min,domitorio_max,precio_m2_pen,tiene_mantenimiento,tiene_estacionamientos,banios
0,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Mariano Carranza al 400\nSanta Beatriz, Lima C...","S/ 2,300 · USD 683",2300.0,683.0,NaN,...,0,61.0,NaN,61.0,2.0,NaN,37.70,0,0,2
1,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Avenida Colonial 3046\nLa Colonial, Lima Cercado","S/ 1,700\n5%",1700.0,NaN,5%,...,0,60.0,NaN,60.0,3.0,NaN,28.33,1,0,2
2,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Parque de la Reserva 736\nLima Cercado, Lima","S/ 2,500",2500.0,NaN,NaN,...,0,75.0,NaN,75.0,3.0,NaN,33.33,0,0,2
3,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Av. Alejandro Bertello Cdra\nPando, Lima Cercado","S/ 1,550",1550.0,NaN,NaN,...,0,32.0,NaN,32.0,1.0,NaN,48.44,1,0,1
4,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Av. Oscar R. Benavides\nLima Cercado, Lima","S/ 1,500",1500.0,NaN,NaN,...,0,50.0,NaN,50.0,2.0,NaN,30.00,1,0,2


In [300]:
data_analisis['distrito_categoria'] = data_analisis['zona'].astype('category')

In [301]:
data_analisis['distrito_categoria'].value_counts()

distrito_categoria
Lima Top        31327
Lima Moderna     8612
Lima Sur         3841
Lima Centro      2100
Lima Norte       2068
Lima Este        1736
Name: count, dtype: int64

In [302]:
# Se crean columnas categóricas para los rangos de precio y área, que se usarán en los filtros de las pestañas.
data_analisis['precio_pen'] = pd.to_numeric(data_analisis['precio_pen'], errors='coerce')
data_analisis['precio_usd'] = pd.to_numeric(data_analisis['precio_usd'], errors='coerce')
data_analisis['area'] = pd.to_numeric(data_analisis['area'], errors='coerce')

In [303]:
data_analisis[data_analisis["operacion"]=="alquiler"]["precio_pen"].describe()

count    7.617000e+03
mean     8.203685e+03
std      8.032473e+04
min      4.000000e+02
25%      2.100000e+03
50%      2.800000e+03
75%      4.358000e+03
max      2.500000e+06
Name: precio_pen, dtype: float64

In [305]:
data_analisis[data_analisis["operacion"]=="venta"]["precio_usd"].describe()

count    4.206700e+04
mean     4.320637e+05
std      2.183362e+06
min      2.941176e-01
25%      1.435000e+05
50%      2.400000e+05
75%      4.500000e+05
max      2.160000e+08
Name: precio_usd, dtype: float64

In [306]:
# Bins y labels para precio de venta (Soles)
bins_alquiler = [-1, 1500, 2500, 3500, 4500, float('inf')]
labels_alquiler = ["Hasta S/1.5k", "De S/1.5k a S/2.5k", "De S/2.5 a S/3.5k", "De S/3.5k a S/4.5k", "De S/4.5k a más"]
data_analisis['precio_alquiler_agp'] = pd.cut(data_analisis[data_analisis["operacion"]=="alquiler"]["precio_pen"], bins=bins_alquiler, labels=labels_alquiler, right=False)

# Bins y labels para precio de venta (Dólares)
bins_venta = [-1, 50000, 100000, 200000, 500000, float('inf')]
labels_venta = ["Hasta $ 50k", "De $ 50k a $ 100k", "De $ 100k a $ 200k", "De $ 200k a $ 500k", "De $ 500k a más"]
data_analisis['precio_venta_agp'] = pd.cut(data_analisis[data_analisis["operacion"]=="venta"]['precio_usd'], bins=bins_venta, labels=labels_venta, right=False)

# Bins y labels para área (m²)
bins_area = [-1, 50, 100, 200, 300, float('inf')]
labels_area = ["Hasta 50m2", "De 50m2 a 100m2", "De 100m2 a 200m2", "De 200m2 a 300m2", "De 300m2 a más"]
data_analisis['area_agp'] = pd.cut(data_analisis['area'], bins=bins_area, labels=labels_area, right=False)

## Estacionamiento
data_analisis["estacionamiento_gp"] = data_analisis["estacionamientos"].apply(lambda x: "Si" if x > 0 else "No")

In [307]:
data_analisis["distrito_oficial"].value_counts()

distrito_oficial
Santiago de Surco          8841
Miraflores                 8739
San Isidro                 5613
La Molina                  3490
San Borja                  2506
San Miguel                 2434
Barranco                   2016
Surquillo                  1624
Chorrillos                 1464
Jesús María                1292
Magdalena                  1227
Lima Cercado               1076
Pueblo Libre               1046
Lince                       966
Ate Vitarte                 719
Los Olivos                  579
San Martin de Porres        534
Punta Hermosa               525
La Victoria                 487
San Juan de Lurigancho      472
San Bartolo                 404
Comas                       386
San Juan de Miraflores      359
Carabayllo                  271
Lurín                       254
Breña                       253
Rimac                       190
Independencia               173
Santa Maria del Mar         155
Villa El Salvador           154
Pachacamac             

In [308]:
data_analisis.columns

Index(['fuente', 'inmueble', 'operacion', 'distrito_oficial', 'zona',
       'direccion', 'precio', 'soles', 'dolares', 'variacion', 'mantenimiento',
       'caracteristica', 'area', 'dormitorio', 'baños', 'estacionamientos',
       'detalle', 'enlace', 'fecha', 'file_origin', 'distrito', 'precio_pen',
       'precio_usd', 'sin_precio', 'area_min', 'area_max', 'area_promedio',
       'domitorio_min', 'domitorio_max', 'precio_m2_pen',
       'tiene_mantenimiento', 'tiene_estacionamientos', 'banios',
       'distrito_categoria', 'precio_alquiler_agp', 'precio_venta_agp',
       'area_agp', 'estacionamiento_gp'],
      dtype='object')

In [309]:
data_analisis.head()

,fuente,inmueble,operacion,distrito_oficial,zona,direccion,precio,soles,dolares,variacion,...,domitorio_max,precio_m2_pen,tiene_mantenimiento,tiene_estacionamientos,banios,distrito_categoria,precio_alquiler_agp,precio_venta_agp,area_agp,estacionamiento_gp
0,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Mariano Carranza al 400\nSanta Beatriz, Lima C...","S/ 2,300 · USD 683",2300.0,683.0,NaN,...,NaN,37.70,0,0,2,Lima Centro,De S/1.5k a S/2.5k,NaN,De 50m2 a 100m2,No
1,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Avenida Colonial 3046\nLa Colonial, Lima Cercado","S/ 1,700\n5%",1700.0,NaN,5%,...,NaN,28.33,1,0,2,Lima Centro,De S/1.5k a S/2.5k,NaN,De 50m2 a 100m2,No
2,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Parque de la Reserva 736\nLima Cercado, Lima","S/ 2,500",2500.0,NaN,NaN,...,NaN,33.33,0,0,2,Lima Centro,De S/2.5 a S/3.5k,NaN,De 50m2 a 100m2,No
3,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Av. Alejandro Bertello Cdra\nPando, Lima Cercado","S/ 1,550",1550.0,NaN,NaN,...,NaN,48.44,1,0,1,Lima Centro,De S/1.5k a S/2.5k,NaN,Hasta 50m2,No
4,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Av. Oscar R. Benavides\nLima Cercado, Lima","S/ 1,500",1500.0,NaN,NaN,...,NaN,30.00,1,0,2,Lima Centro,De S/1.5k a S/2.5k,NaN,De 50m2 a 100m2,No


In [ ]:
cols_final = [
    'fuente','inmueble','operacion', 'distrito', 'zona','direccion',
    'precio','soles','dolares','precio_pen','precio_usd','variacion',
    'mantenimiento','tiene_mantenimiento','caracteristica',
    'area','area_min','area_max','area_promedio', 'area_agp', 'precio_m2_pen', 'precio_venta_agp', 'precio_alquiler_agp',
    'dormitorio','domitorio_min','domitorio_max','banios', 'estacionamientos','tiene_estacionamientos', 'estacionamiento_gp',
    'detalle','enlace','fecha'
]
data_analisis = data_analisis[cols_final].copy()# Orden final


In [311]:
data_analisis.head(100)

,fuente,inmueble,operacion,distrito,zona,direccion,precio,soles,dolares,precio_pen,...,dormitorio,domitorio_min,domitorio_max,banios,estacionamientos,tiene_estacionamientos,estacionamiento_gp,detalle,enlace,fecha
0,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Mariano Carranza al 400\nSanta Beatriz, Lima C...","S/ 2,300 · USD 683",2300.0,683.0,2300.0,...,2,2.0,NaN,2,NaN,0,No,Departamento de estreno con Balcón y Excelente...,https://www.adondevivir.com/propiedades/clasif...,2025-12-23
1,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Avenida Colonial 3046\nLa Colonial, Lima Cercado","S/ 1,700\n5%",1700.0,NaN,1700.0,...,3,3.0,NaN,2,NaN,0,No,Se Alquila Departamento bien iluminado con Bal...,https://www.adondevivir.com/propiedades/clasif...,2025-12-23
2,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Parque de la Reserva 736\nLima Cercado, Lima","S/ 2,500",2500.0,NaN,2500.0,...,3,3.0,NaN,2,NaN,0,No,Contacto: malena peláez // cel: 993*306*450. S...,https://www.adondevivir.com/propiedades/clasif...,2025-12-23
3,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Av. Alejandro Bertello Cdra\nPando, Lima Cercado","S/ 1,550",1550.0,NaN,1550.0,...,1,1.0,NaN,1,NaN,0,No,Alquiler de Departamento en Lima Cercado - Urb...,https://www.adondevivir.com/propiedades/clasif...,2025-12-23
4,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Av. Oscar R. Benavides\nLima Cercado, Lima","S/ 1,500",1500.0,NaN,1500.0,...,2,2.0,NaN,2,NaN,0,No,"Con un área de 50m2, Ubicado en el Piso 14 con...",https://www.adondevivir.com/propiedades/clasif...,2025-12-23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,adondevivir,departamentos,alquiler,La Victoria,Lima Centro,Avenida Paseo de la Republica 2199\nLa Victori...,"S/ 1,900",1900.0,NaN,1900.0,...,1,1.0,NaN,1,NaN,0,No,Se alquila moderno loft de 45 m² en el piso 27...,https://www.adondevivir.com/propiedades/clasif...,2025-12-23
97,adondevivir,departamentos,alquiler,La Victoria,Lima Centro,"ON APARTMENTS\nLa Victoria, Lima","S/ 2,400 · USD 705",2400.0,705.0,2400.0,...,3,3.0,NaN,2,1.0,1,Si,Alquilo moderno departamento en Santa Catalina...,https://www.adondevivir.com/propiedades/clasif...,2025-12-23
98,adondevivir,departamentos,alquiler,La Victoria,Lima Centro,"Javier Prado 1071\nCorpac, La Victoria","S/ 2,200",2200.0,NaN,2200.0,...,1,1.0,NaN,1,NaN,0,No,Súper Céntrico ubicación inmejorable departame...,https://www.adondevivir.com/propiedades/clasif...,2025-12-23
99,adondevivir,departamentos,alquiler,La Victoria,Lima Centro,"Audacity\nSanta Catalina, La Victoria","S/ 1,750",1750.0,NaN,1750.0,...,NaN,NaN,NaN,1,NaN,0,No,"Alquilo Exclusivo Loft en Javier Prado! S/ 1, ...",https://www.adondevivir.com/propiedades/clasif...,2025-12-23


In [312]:
data_analisis[data_analisis["dormitorio"]==20]

,fuente,inmueble,operacion,distrito,zona,direccion,precio,soles,dolares,precio_pen,...,dormitorio,domitorio_min,domitorio_max,banios,estacionamientos,tiene_estacionamientos,estacionamiento_gp,detalle,enlace,fecha
8053,adondevivir,casas,venta,Santa Anita,Lima Este,"avenida los cipreses\nLos Ficus, Santa Anita","S/ 1,561,500 · USD 450,000",1561500.0,450000.0,1561500.0,...,20.0,20.0,NaN,8,NaN,0,No,¡Gran oportunidad de inversión en una de las z...,https://www.adondevivir.com/propiedades/clasif...,2025-12-24
8144,adondevivir,casas,venta,San Juan De Lurigancho,Lima Este,"Coyllur\nZarate, San Juan de Lurigancho","S/ 965,000 · USD 287,000",965000.0,287000.0,965000.0,...,20.0,20.0,NaN,8,1.0,1,Si,Quieres que tu casa te genere activos? Distrib...,https://www.adondevivir.com/propiedades/clasif...,2025-12-24
8718,adondevivir,casas,venta,San Martin De Porres,Lima Norte,"FILADELFIA 2882\nSan Martín de Porres, Lima","S/ 893,000 · USD 265,000",893000.0,265000.0,893000.0,...,20.0,20.0,NaN,10,1.0,1,Si,Ideal para inversión – av. Perú con av. Univer...,https://www.adondevivir.com/propiedades/clasif...,2025-12-24
8726,adondevivir,casas,venta,San Martin De Porres,Lima Norte,"Calle Eduardo Girardo N° 191 -193, SAN Martín ...","S/ 1,645,000 · USD 470,000",1645000.0,470000.0,1645000.0,...,20.0,20.0,NaN,8,1.0,1,Si,Casa multifamiliar en venta ubicada en San Mar...,https://www.adondevivir.com/propiedades/clasif...,2025-12-24
8868,adondevivir,casas,venta,San Martin De Porres,Lima Norte,"FILADELFIA 2882\nSan Martín de Porres, Lima","S/ 893,000 · USD 265,000",893000.0,265000.0,893000.0,...,20.0,20.0,NaN,10,1.0,1,Si,Ideal para inversión – av. Perú con av. Univer...,https://www.adondevivir.com/propiedades/clasif...,2025-12-24
8869,adondevivir,casas,venta,San Martin De Porres,Lima Norte,"Calle Eduardo Girardo N° 191 -193, SAN Martín ...","S/ 1,645,000 · USD 470,000",1645000.0,470000.0,1645000.0,...,20.0,20.0,NaN,8,1.0,1,Si,Casa multifamiliar en venta ubicada en San Mar...,https://www.adondevivir.com/propiedades/clasif...,2025-12-24
8965,adondevivir,casas,venta,San Martin De Porres,Lima Norte,urbanizacion virgen de fatima\nSan Martín de P...,"S/ 1,120,000 · USD 320,000",1120000.0,320000.0,1120000.0,...,20.0,20.0,NaN,11,NaN,0,No,Excelente oportunidad de inversión! Venta de c...,https://www.adondevivir.com/propiedades/clasif...,2025-12-24
9218,adondevivir,casas,venta,Chorrillos,Lima Sur,"Qxvj+x5c, Chorrillos 15067, Perú\nChorrillos, ...","S/ 1,260,000 · USD 360,000",1260000.0,360000.0,1260000.0,...,20.0,20.0,NaN,19,6.0,1,Si,Inversión rentable en chorrillos¡Oportunidad d...,https://www.adondevivir.com/propiedades/clasif...,2025-12-24
13197,urbania,casas,venta,Santa Anita,Lima Este,"avenida los cipreses\nLos Ficus, Santa Anita","S/ 1,561,500 · USD 450,000",1561500.0,450000.0,1561500.0,...,20.0,20.0,NaN,8,NaN,0,No,¡Gran oportunidad de inversión en una de las z...,https://urbania.pe/inmueble/clasificado/veclca...,2025-12-24
13250,urbania,casas,venta,San Juan De Lurigancho,Lima Este,"Coyllur\nZarate, San Juan de Lurigancho","S/ 965,000 · USD 287,000",965000.0,287000.0,965000.0,...,20.0,20.0,NaN,8,1.0,1,Si,Quieres que tu casa te genere activos? Distrib...,https://urbania.pe/inmueble/clasificado/veclca...,2025-12-24


In [313]:
zonas_lima = {
        'Lima Top': ['Miraflores', 'San Isidro', 'La Molina', 'Santiago De Surco', 'San Borja', 'Barranco'],
        'Lima Moderna': ['Jesús María', 'Lince', 'Magdalena', 'San Miguel', 'Pueblo Libre', 'Surquillo'],
        'Lima Centro': ['Lima Cercado', 'Breña', 'La Victoria', 'Rimac', 'San Luis'],
        'Lima Este': ['Ate Vitarte', 'Cieneguilla', 'Chaclacayo', 'Chosica Lurigancho', 'Santa Anita', 'El Agustino', 'San Juan De Lurigancho'],
        'Lima Norte': ['Carabayllo', 'Comas', 'Independencia', 'Los Olivos', 'Puente Piedra', 'San Martin De Porres', 'Ancón', 'Santa Rosa'],
        'Lima Sur': ['Chorrillos', 'Lurín', 'Pachacamac', 'San Juan De Miraflores', 'Villa El Salvador', 'Villa Maria Del Triunfo', 'Pucusana', 'Punta Hermosa', 'Punta Negra', 'San Bartolo', 'Santa Maria Del Mar']
    }

distrito_a_zona = {distrito: zona for zona, distritos_en_zona in zonas_lima.items() for distrito in distritos_en_zona}

In [314]:
data_analisis["zona"] = data_analisis["distrito"].map(distrito_a_zona).fillna('Otra Zona')

In [315]:
data_analisis[data_analisis["zona"]=="Otra Zona"]["distrito"].unique()

array([], dtype=object)

In [316]:
data_analisis.columns

Index(['fuente', 'inmueble', 'operacion', 'distrito', 'zona', 'direccion',
       'precio', 'soles', 'dolares', 'precio_pen', 'precio_usd', 'variacion',
       'mantenimiento', 'tiene_mantenimiento', 'caracteristica', 'area',
       'area_min', 'area_max', 'area_promedio', 'area_agp', 'precio_m2_pen',
       'precio_venta_agp', 'precio_alquiler_agp', 'dormitorio',
       'domitorio_min', 'domitorio_max', 'banios', 'estacionamientos',
       'tiene_estacionamientos', 'estacionamiento_gp', 'detalle', 'enlace',
       'fecha'],
      dtype='object')

In [317]:
data_analisis["dormitorio"].describe()

count     48472
unique       61
top           3
freq      19284
Name: dormitorio, dtype: object

In [318]:
data_analisis["domitorio_min"].describe()

count    48472.000000
mean         3.030678
std          1.713840
min          1.000000
25%          2.000000
50%          3.000000
75%          3.000000
max         55.000000
Name: domitorio_min, dtype: float64

In [319]:
data_analisis["dormitorios"] = pd.cut(data_analisis['domitorio_min'],
                                    bins=[1, 2, 3, 4, 5, float('inf')],
                                    labels=['1 Dormitorio', '2 Dormitorios', '3 Dormitorios', '4 Dormitorios', '5 o más Dormitorios'],
                                    right=False)

In [320]:
data_analisis[["dormitorio", "domitorio_min", "dormitorios", "detalle"]].head(20)

,dormitorio,domitorio_min,dormitorios,detalle
0,2,2.0,2 Dormitorios,Departamento de estreno con Balcón y Excelente...
1,3,3.0,3 Dormitorios,Se Alquila Departamento bien iluminado con Bal...
2,3,3.0,3 Dormitorios,Contacto: malena peláez // cel: 993*306*450. S...
3,1,1.0,1 Dormitorio,Alquiler de Departamento en Lima Cercado - Urb...
4,2,2.0,2 Dormitorios,"Con un área de 50m2, Ubicado en el Piso 14 con..."
5,3,3.0,3 Dormitorios,Alquilo departamento de estreno en 2do piso en...
6,3,3.0,3 Dormitorios,Alquilo moderno departamento en lima cercado. ...
7,3,3.0,3 Dormitorios,Alquiler de Departamento de Estreno en Condomi...
8,3,3.0,3 Dormitorios,Departamento de 73m2 muy bien distribuidos. Sa...
9,3,3.0,3 Dormitorios,Departamento de Estreno en Alquiler – 3 Dormit...


In [321]:
data_analisis["dormitorios"].value_counts()

dormitorios
3 Dormitorios          21411
2 Dormitorios          10887
4 Dormitorios           6531
1 Dormitorio            5140
5 o más Dormitorios     4503
Name: count, dtype: int64

In [ ]:
def detected_atipic_price_alquiler(distrito):
    
    data_distrito = data_analisis[(data_analisis["distrito"] == distrito) & 
                            (data_analisis["operacion"]== "alquiler")
                            ] 
    
    metrica = data_distrito["precio_pen"].median()
    q75 = data_distrito["precio_pen"].quantile(0.75)
    q25 = data_distrito["precio_pen"].quantile(0.25)
    iqr = q75 - q25
    umbral_superior = q75 + 1.5 * iqr
    umbral_inferior = q25 - 1.5 * iqr
    
    data_filtrada = data_distrito[(data_distrito["precio_pen"]<=umbral_inferior) | (data_distrito["precio_pen"]>=umbral_superior)].copy()
    
    data_filtrada["Atipico_Flag"] = np.where(data_filtrada["precio_pen"]<=umbral_inferior, "Precio Muy Bajo", "Precio Muy Alto").copy()
    
    # data_filtrada["Flag_Operacion"] = data_filtrada["detalle"].str.contains(r"\b(venta|vendo|se vende|en venta)\b", case = False, na=False).astype(int)
    
    data_filtrada["umbral_superior"] = umbral_superior
    data_filtrada["umbral_inferior"] = umbral_inferior
    data_filtrada["mediana"] = metrica
    
    condiciones = [
        (data_filtrada["zona"] == "Lima Top") &
        ((data_filtrada["precio_pen"] - data_filtrada["umbral_superior"]) < 20000),
    
        (data_filtrada["zona"].isin(["Lima Moderna", "Lima Centro"])) &
        ((data_filtrada["precio_pen"] - data_filtrada["umbral_superior"]) < 2000),
    
        (data_filtrada["zona"].isin(["Lima Este", "Lima Norte", "Lima Sur"])) &
        ((data_filtrada["precio_pen"] - data_filtrada["umbral_superior"]) < 1000),
    ]
    
    etiquetas = ["alquiler", "alquiler", "alquiler"]
    
    data_filtrada["casos"] = np.select(condiciones, etiquetas, default="quitar")
    
    return data_filtrada[["operacion", "precio_pen", "precio_usd", "mediana", "umbral_superior", "umbral_inferior", "Atipico_Flag", "casos", "direccion","detalle", "enlace"]]

In [323]:
distritos = data_analisis["distrito"].unique()

data_quitar = pd.DataFrame()
for distrito in distritos:
    
    datos_atipicos = detected_atipic_price_alquiler(distrito)
    datos_atipicos = datos_atipicos[datos_atipicos["casos"] == "quitar"].copy()
    # if not datos_atipicos.empty:
        # print(f"\nDistrito: {distrito} - Registros Atípicos: {len(datos_atipicos)}")
    data_quitar = pd.concat([data_quitar, datos_atipicos], ignore_index=True) 
    
    
data_analisis_temp = data_analisis.merge(
    data_quitar[["enlace", "casos"]],
    on = "enlace",
    how = "left"
    
)

data_analisis_limpia = data_analisis_temp[data_analisis_temp["casos"] != "quitar"]


In [325]:
print("\nData Final creada satisfactiramente!")

# Export
out_path = DATA_OUT / "data_dondevivir_analisis.csv"
data_analisis_limpia.to_csv(out_path, sep="|", index=False)
print("\nData Final creada satisfactoriamente ->", out_path)


Data Final creada satisfactiramente!

Data Final creada satisfactoriamente -> C:\Users\PC\Desktop\Proyectos\Proyectos_Py\1.Analisis Vivienda\Analisis_Vivienda\data\processed\data_dondevivir_analisis.csv


In [270]:
data_analisis_limpia.to_excel(DATA_OUT/ "data_adondevivir_analisis.xlsx", index=False)

In [326]:
data_analisis_limpia[(data_analisis_limpia["distrito"]=="Miraflores")&(data_analisis_limpia["operacion"]=="venta")]

,fuente,inmueble,operacion,distrito,zona,direccion,precio,soles,dolares,precio_pen,...,domitorio_max,banios,estacionamientos,tiene_estacionamientos,estacionamiento_gp,detalle,enlace,fecha,dormitorios,casos
8419,adondevivir,casas,venta,Miraflores,Lima Top,"Calle Italia 260\nMiraflores, Lima","S/ 638,000 · USD 183,000",638000.0,183000.0,638000.0,...,3.0,<NA>,NaN,0,No,"Rodas combina lujo, comodidad y modernidad en ...",https://www.adondevivir.com/propiedades/proyec...,2025-12-24,1 Dormitorio,NaN
8420,adondevivir,casas,venta,Miraflores,Lima Top,Calle Tnt Crn Gustavo Escudero Otero 228 Miraf...,"S/ 1,948,800 · USD 580,000",1948800.0,580000.0,1948800.0,...,NaN,5,5.0,1,Si,Descripción de la propiedad mínimo 130 palabra...,https://www.adondevivir.com/propiedades/clasif...,2025-12-24,5 o más Dormitorios,NaN
8423,adondevivir,casas,venta,Miraflores,Lima Top,"CALLE FRANCIA\nMiraflores, Lima","S/ 2,200,000 · USD 658,000",2200000.0,658000.0,2200000.0,...,NaN,4,2.0,1,Si,"Vive A un paso del malecón cisneros, calle tra...",https://www.adondevivir.com/propiedades/clasif...,2025-12-24,4 Dormitorios,NaN
9607,adondevivir,casas,venta,Miraflores,Lima Top,"Miguel Aljovin al 385, Miraflores\nMiraflores,...","S/ 2,544,619 · USD 690,000",2544619.0,690000.0,2544619.0,...,NaN,2,2.0,1,Si,"Linda y acogedora casa zona tranquila, excelen...",https://www.adondevivir.com/propiedades/clasif...,2025-12-24,3 Dormitorios,NaN
9608,adondevivir,casas,venta,Miraflores,Lima Top,"CALLE SALAVERRY\nMiraflores, Lima","S/ 1,490,000 · USD 445,000",1490000.0,445000.0,1490000.0,...,NaN,6,1.0,1,Si,"Se vende casa muy acogedora, 160 m2. De terren...",https://www.adondevivir.com/propiedades/clasif...,2025-12-24,4 Dormitorios,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42579,urbania,departamentos,venta,Miraflores,Lima Top,"Calle José Gonzales 640 - 644\nMiraflores, Lima","S/ 1,010,960 · USD 299,990",1010960.0,299990.0,1010960.0,...,NaN,3,1.0,1,Si,Dúplex de estreno en una de la mejores zonas d...,https://urbania.pe/inmueble/clasificado/veclap...,2025-12-22,2 Dormitorios,NaN
42580,urbania,departamentos,venta,Miraflores,Lima Top,"Av Mendiburu N° 12xx, Dpto. 306 AL 1206\nMiraf...","S/ 562,500 · USD 150,000",562500.0,150000.0,562500.0,...,NaN,2,1.0,1,Si,"Edificio en Miraflores, 12 pisos, 66 departame...",https://urbania.pe/inmueble/clasificado/veclap...,2025-12-22,2 Dormitorios,NaN
42581,urbania,departamentos,venta,Miraflores,Lima Top,"Calle Santa Isabel N° 1xx, Dpto. 305, 705\nMir...","S/ 930,800",930800.0,NaN,930800.0,...,NaN,2,NaN,0,No,"Edificio de 9 pisos, 4 sótanos, 83 departament...",https://urbania.pe/inmueble/clasificado/veclap...,2025-12-22,2 Dormitorios,NaN
42582,urbania,departamentos,venta,Miraflores,Lima Top,"Calle FRANCIA 551 - 705\nMiraflores, Lima","S/ 1,209,600 · USD 360,000",1209600.0,360000.0,1209600.0,...,NaN,2,1.0,1,Si,Moderno Dúplex vista Externa en Venta – 212 m²...,https://urbania.pe/inmueble/clasificado/veclap...,2025-12-22,3 Dormitorios,NaN


## Reasignacion de Venta o Alquiler, segun el precio

In [182]:
data_analisis[(data_analisis["distrito"]=="Magdalena")&(data_analisis["operacion"] == "alquiler")]

,fuente,inmueble,operacion,distrito,zona,direccion,precio,soles,dolares,precio_pen,...,estacionamientos,tiene_estacionamientos,estacionamiento_gp,detalle,enlace,fecha,dormitorios,casos_x,casos_y,casos
547,adondevivir,departamentos,alquiler,Magdalena,Lima Moderna,"FRANCISCO GRAÑA\nMagdalena, Lima","S/ 3,300 · USD 985",3300.0,985.0,3300.0,...,1.0,1,Si,Magdalena del mar · alquiler premium. Departam...,https://www.adondevivir.com/propiedades/clasif...,2025-12-31,3 Dormitorios,NaN,NaN,NaN
548,adondevivir,departamentos,alquiler,Magdalena,Lima Moderna,"Jr SALAVERRY\nMagdalena, Lima","S/ 2,000 · USD 570",2000.0,570.0,2000.0,...,NaN,0,No,"Dpto sin amoblar en piso 2, con vista interior...",https://www.adondevivir.com/propiedades/clasif...,2025-12-31,3 Dormitorios,NaN,NaN,NaN
549,adondevivir,departamentos,alquiler,Magdalena,Lima Moderna,"Avenida Salaverry 2900\nMagdalena, Lima","S/ 3,182 · USD 950",3182.0,950.0,3182.0,...,1.0,1,Si,Alquiler de acogedor departamento ubicado entr...,https://www.adondevivir.com/propiedades/clasif...,2025-12-31,2 Dormitorios,NaN,NaN,NaN
550,adondevivir,departamentos,alquiler,Magdalena,Lima Moderna,"Av. Gonzales Prada al 400\nMagdalena, Lima","S/ 1,700\n5%",1700.0,NaN,1700.0,...,NaN,0,No,Alquiler de departmento de 45 M2 en av. Gonzal...,https://www.adondevivir.com/propiedades/clasif...,2025-12-31,1 Dormitorio,NaN,NaN,NaN
551,adondevivir,departamentos,alquiler,Magdalena,Lima Moderna,Av. Javier Prado Oeste 140 - 160 Magdalena del...,"S/ 2,200",2200.0,NaN,2200.0,...,NaN,0,No,Moderno dpto a un paso ubicado en zona céntric...,https://www.adondevivir.com/propiedades/clasif...,2025-12-31,2 Dormitorios,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4486,urbania,departamentos,alquiler,Magdalena,Lima Moderna,"Avenida Antonio Miró Quesada 635\nMagdalena, Lima","S/ 2,300",2300.0,NaN,2300.0,...,1.0,1,Si,"Moderno y acogedor departamento de 58 m², ubic...",https://urbania.pe/inmueble/clasificado/alclap...,2025-12-31,2 Dormitorios,NaN,NaN,NaN
4487,urbania,departamentos,alquiler,Magdalena,Lima Moderna,"Avenida Pershing 820\nMagdalena, Lima","S/ 2,800 · USD 800",2800.0,800.0,2800.0,...,NaN,0,No,Alquiler de Departamento de Estreno Amoblado –...,https://urbania.pe/inmueble/clasificado/alclap...,2025-12-31,1 Dormitorio,NaN,NaN,NaN
4488,urbania,departamentos,alquiler,Magdalena,Lima Moderna,"Av. Alberto del Campo 414\nMagdalena, Lima","S/ 2,500 · USD 744",2500.0,744.0,2500.0,...,NaN,0,No,En esta oportunidad tenemos un departamento es...,https://urbania.pe/inmueble/clasificado/alclap...,2025-12-31,2 Dormitorios,NaN,NaN,NaN
4489,urbania,departamentos,alquiler,Magdalena,Lima Moderna,jr Juan de Aliaga 250 Magdalena de Mar\nSan Fe...,"S/ 4,500",4500.0,NaN,4500.0,...,1.0,1,Si,Alquiler | dúplex frente al lima cricket. Uso ...,https://urbania.pe/inmueble/clasificado/alclap...,2025-12-31,5 o más Dormitorios,NaN,NaN,NaN


In [162]:
# stats_alquiler = data_analisis[data_analisis["operacion"]=="alquiler"].groupby(['distrito']).agg(
#     conteo   = ('precio_pen', 'count'),
#     promedio = ('precio_pen', 'mean'),
#     mediana  = ('precio_pen', 'median'),
#     minimo   = ('precio_pen', 'min'),
#     maximo   = ('precio_pen', 'max'),
#     p25      = ('precio_pen', lambda x: x.quantile(0.25)),
#     p75      = ('precio_pen', lambda x: x.quantile(0.75)),
# ).reset_index(drop=False)

# stats_alquiler.sort_values(by="conteo", ascending=False).head(50)

In [234]:
def detected_atipic_price(distrito, operacion):
    
    data_distrito = data_analisis[(data_analisis["distrito"] == distrito) & 
                            (data_analisis["operacion"]== operacion)
                            ] 
    
    metrica = data_distrito["precio_pen"].median()
    q75 = data_distrito["precio_pen"].quantile(0.75)
    q25 = data_distrito["precio_pen"].quantile(0.25)
    iqr = q75 - q25
    umbral_superior = q75 + 1.5 * iqr
    umbral_inferior = q25 - 1.5 * iqr
    
    data_filtrada = data_distrito[(data_distrito["precio_pen"]<=umbral_inferior) | (data_distrito["precio_pen"]>=umbral_superior)].copy()
    
    data_filtrada["Atipico_Flag"] = np.where(data_filtrada["precio_pen"]<=umbral_inferior, "Precio Muy Bajo", "Precio Muy Alto").copy()
    
    # data_filtrada["Flag_Operacion"] = data_filtrada["detalle"].str.contains(r"\b(venta|vendo|se vende|en venta)\b", case = False, na=False).astype(int)
    
    data_filtrada["umbral_superior"] = umbral_superior
    data_filtrada["umbral_inferior"] = umbral_inferior
    data_filtrada["mediana"] = metrica
    
    condiciones = [
        (data_filtrada["zona"] == "Lima Top") &
        ((data_filtrada["precio_pen"] - data_filtrada["umbral_superior"]) < 20000),
    
        (data_filtrada["zona"].isin(["Lima Moderna", "Lima Centro"])) &
        ((data_filtrada["precio_pen"] - data_filtrada["umbral_superior"]) < 2000),
    
        (data_filtrada["zona"].isin(["Lima Este", "Lima Norte", "Lima Sur"])) &
        ((data_filtrada["precio_pen"] - data_filtrada["umbral_superior"]) < 1000),
    ]
    
    etiquetas = ["alquiler", "alquiler", "alquiler"]
    
    data_filtrada["casos"] = np.select(condiciones, etiquetas, default="quitar")
    
    return data_filtrada[["operacion", "precio_pen", "precio_usd", "mediana", "umbral_superior", "umbral_inferior", "Atipico_Flag", "casos", "direccion","detalle", "enlace"]]

In [235]:
data_fix = detected_atipic_price("Punta Hermosa", "alquiler")
data_fix.sort_values(by="precio_pen", ascending=False).head(20)

,operacion,precio_pen,precio_usd,mediana,umbral_superior,umbral_inferior,Atipico_Flag,casos,direccion,detalle,enlace
5246,alquiler,101175.0,30000.0,11602.5,73800.0,-34200.0,Precio Muy Alto,quitar,"Malecon Superior\nPunta Hermosa, Lima",Alquiler de departamento dúplex – punta hermos...,https://urbania.pe/inmueble/clasificado/alclap...
1567,alquiler,101175.0,30000.0,11602.5,73800.0,-34200.0,Precio Muy Alto,quitar,"Malecon Superior\nPunta Hermosa, Lima",Alquiler de departamento dúplex – punta hermos...,https://www.adondevivir.com/propiedades/clasif...
1532,alquiler,74100.0,19500.0,11602.5,73800.0,-34200.0,Precio Muy Alto,alquiler,"malecon superior\nPunta Hermosa, Punta Hermosa","Alquiler casa de Lujo frente a playa blanca, p...",https://www.adondevivir.com/propiedades/clasif...
1406,alquiler,74100.0,19500.0,11602.5,73800.0,-34200.0,Precio Muy Alto,alquiler,"malecon superior\nPunta Hermosa, Punta Hermosa","Alquiler casa de Lujo frente a playa blanca, p...",https://www.adondevivir.com/propiedades/clasif...
1583,alquiler,74100.0,19500.0,11602.5,73800.0,-34200.0,Precio Muy Alto,alquiler,"malecon superior\nPunta Hermosa, Punta Hermosa","Alquiler casa de Lujo frente a playa blanca, p...",https://www.adondevivir.com/propiedades/clasif...
5218,alquiler,74100.0,19500.0,11602.5,73800.0,-34200.0,Precio Muy Alto,alquiler,"malecon superior\nPunta Hermosa, Punta Hermosa","Alquiler casa de Lujo frente a playa blanca, p...",https://urbania.pe/inmueble/clasificado/alclap...


In [237]:
distritos = data_analisis["distrito"].unique()

data_quita = pd.DataFrame()
for distrito in distritos:
    
    datos_atipicos = detected_atipic_price(distrito, "alquiler")
    datos_atipicos = datos_atipicos[datos_atipicos["casos"] == "quitar"].copy()
    if not datos_atipicos.empty:
        print(f"\nDistrito: {distrito} - Registros Atípicos: {len(datos_atipicos)}")
    data_quita = pd.concat([data_quita, datos_atipicos], ignore_index=True) 


Distrito: Lima Cercado - Registros Atípicos: 1

Distrito: Jesús María - Registros Atípicos: 2

Distrito: San Isidro - Registros Atípicos: 6

Distrito: San Martin De Porres - Registros Atípicos: 4

Distrito: San Luis - Registros Atípicos: 1

Distrito: San Miguel - Registros Atípicos: 2

Distrito: San Juan De Lurigancho - Registros Atípicos: 2

Distrito: Santa Anita - Registros Atípicos: 2

Distrito: Lince - Registros Atípicos: 2

Distrito: Magdalena - Registros Atípicos: 5

Distrito: Chorrillos - Registros Atípicos: 12

Distrito: Lurín - Registros Atípicos: 5

Distrito: Punta Hermosa - Registros Atípicos: 2

Distrito: Santiago De Surco - Registros Atípicos: 4

Distrito: Santa Maria Del Mar - Registros Atípicos: 5

Distrito: Punta Negra - Registros Atípicos: 2

Distrito: San Bartolo - Registros Atípicos: 5

Distrito: Miraflores - Registros Atípicos: 16

Distrito: Barranco - Registros Atípicos: 5


In [239]:
data_analisis_temp = data_analisis.merge(
    data_quita[["enlace", "casos"]],
    on = "enlace",
    how = "left"
)

data_analisis_limpia = data_analisis_temp[data_analisis_temp["casos"] != "quitar"]

In [226]:
stats_alquiler = data_analisis_limpia[data_analisis_limpia["operacion"]=="alquiler"].groupby(['distrito']).agg(
    conteo   = ('precio_pen', 'count'),
    promedio = ('precio_pen', 'mean'),
    mediana  = ('precio_pen', 'median'),
    minimo   = ('precio_pen', 'min'),
    maximo   = ('precio_pen', 'max'),
    p25      = ('precio_pen', lambda x: x.quantile(0.25)),
    p75      = ('precio_pen', lambda x: x.quantile(0.75)),
).reset_index(drop=False)

stats_alquiler.sort_values(by="conteo", ascending=False).head(50)

,distrito,conteo,promedio,mediana,minimo,maximo,p25,p75
18,Miraflores,1903,4862.222281,3707.0,1150.0,28000.0,2900.00,5600.00
28,San Isidro,962,6226.092516,4785.0,1400.0,25160.0,3100.00,8000.00
37,Santiago De Surco,754,4035.281167,3100.0,800.0,29360.0,2200.00,5049.25
2,Barranco,624,3877.751603,2590.0,1300.0,22800.0,2200.00,3262.50
33,San Miguel,339,2086.858407,2100.0,550.0,3898.0,1700.00,2500.00
10,Jesús María,259,2573.409266,2500.0,1200.0,4500.0,2190.00,2993.00
38,Surquillo,255,2329.184314,2300.0,1000.0,3800.0,2025.00,2600.00
14,Lince,239,2529.301255,2500.0,850.0,4875.0,2200.00,2800.00
27,San Borja,225,3971.057778,3200.0,1000.0,12920.0,2700.00,4700.00
17,Magdalena,214,2729.401869,2500.0,980.0,5680.0,2300.00,3000.00


Caso Venta

In [344]:
stats_venta = data_analisis[(data_analisis["operacion"]=="venta") & (data_analisis["inmueble"]=="departamentos")].groupby(['distrito']).agg(
    conteo   = ('precio_usd', 'count'),
    promedio = ('precio_usd', 'mean'),
    mediana  = ('precio_usd', 'median'),
    minimo   = ('precio_usd', 'min'),
    maximo   = ('precio_usd', 'max'),
    p25      = ('precio_usd', lambda x: x.quantile(0.25)),
    p75      = ('precio_usd', lambda x: x.quantile(0.75)),
).reset_index(drop=False)

stats_venta.sort_values(by="conteo", ascending=False).head(50)

,distrito,conteo,promedio,mediana,minimo,maximo,p25,p75
38,Santiago De Surco,6421,3.034312e+05,255294.117647,0.294118,1.777700e+06,185000.000000,360000.000000
19,Miraflores,6229,3.297896e+05,268500.000000,0.294118,2.500000e+06,207161.000000,375000.000000
29,San Isidro,3936,6.521158e+05,403038.000000,0.294118,2.160000e+08,248764.500000,604528.000000
34,San Miguel,1904,3.351353e+05,118588.235294,1.000000,1.333530e+08,99830.000000,147014.500000
28,San Borja,1798,3.348427e+05,295000.000000,1.000000,1.900000e+06,220000.000000,390000.000000
39,Surquillo,1324,1.435798e+05,135588.235294,720.000000,3.800000e+05,110000.000000,173080.882353
2,Barranco,1323,3.376176e+05,185000.000000,750.000000,3.850000e+06,135000.000000,286500.000000
11,Jesús María,977,1.646516e+05,150000.000000,62000.000000,4.030000e+05,125714.000000,185164.000000
18,Magdalena,917,1.791680e+05,160000.000000,780.000000,6.350000e+05,129880.882353,215000.000000
6,Chorrillos,854,1.466735e+05,110000.000000,45000.000000,1.300000e+06,85000.000000,154814.691176


In [339]:
data_analisis[(data_analisis["distrito"]=="Miraflores") &
            (data_analisis["operacion"]=="venta")
            ]["precio_usd"].min()

np.float64(0.29411764705882354)

In [ ]:
def detected_atipic_price_venta(distrito, operacion):
    
    data_distrito = data_analisis[(data_analisis["distrito"] == distrito) & 
                            (data_analisis["operacion"]== operacion)
                            ] 
    
    metrica = data_distrito["precio_usd"].median()
    q75 = data_distrito["precio_usd"].quantile(0.75)
    q25 = data_distrito["precio_usd"].quantile(0.25)
    iqr = q75 - q25
    umbral_superior = q75 + 1.5 * iqr
    umbral_inferior = q25 - 1.5 * iqr
    
    data_filtrada = data_distrito[(data_distrito["precio_usd"]<=umbral_inferior) | (data_distrito["precio_usd"]>=umbral_superior)].copy()
    
    data_filtrada["Atipico_Flag"] = np.where(data_filtrada["precio_usd"]<=umbral_inferior, "Precio Muy Bajo", "Precio Muy Alto").copy()
    
    # data_filtrada["Flag_Operacion"] = data_filtrada["detalle"].str.contains(r"\b(venta|vendo|se vende|en venta)\b", case = False, na=False).astype(int)
    
    data_filtrada["umbral_superior"] = umbral_superior
    data_filtrada["umbral_inferior"] = umbral_inferior
    data_filtrada["mediana"] = metrica
    
    # condiciones = [
    #     (data_filtrada["zona"] == "Lima Top") &
    #     ((data_filtrada["precio_usd"] - data_filtrada["umbral_superior"]) < 20000),
    
    #     (data_filtrada["zona"].isin(["Lima Moderna", "Lima Centro"])) &
    #     ((data_filtrada["precio_usd"] - data_filtrada["umbral_superior"]) < 2000),
    
    #     (data_filtrada["zona"].isin(["Lima Este", "Lima Norte", "Lima Sur"])) &
    #     ((data_filtrada["precio_usd"] - data_filtrada["umbral_superior"]) < 1000),
    # ]
    
    # etiquetas = ["venta", "venta", "venta"]
    
    # data_filtrada["casos"] = np.select(condiciones, etiquetas, default="quitar")
    
    return data_filtrada[["operacion", "precio_pen", "precio_usd", "mediana", "umbral_superior", "umbral_inferior", "Atipico_Flag", "casos", "direccion","detalle", "enlace"]]

In [329]:
data_prueba = detected_atipic_price_venta("Miraflores", "venta")
data_prueba

,operacion,precio_pen,precio_usd,mediana,umbral_superior,umbral_inferior,Atipico_Flag,casos,direccion,detalle,enlace
9720,venta,4433950.0,1249000.0,280000.0,755522.058824,-112536.764706,Precio Muy Alto,quitar,"Ciro Alegría miraflores\nMiraflores, Lima","Ubicada a solo una cuadra de parque, iglesia S...",https://www.adondevivir.com/propiedades/clasif...
9722,venta,2852640.0,849000.0,280000.0,755522.058824,-112536.764706,Precio Muy Alto,quitar,"Avenida Roca y Boloña al 1200\nMiraflores, Lima",Consultar por la asesora Mirtha Veliz 1. 1. 4....,https://www.adondevivir.com/propiedades/clasif...
9723,venta,3175380.0,897000.0,280000.0,755522.058824,-112536.764706,Precio Muy Alto,quitar,"LAS Fresas al 500\nAurora, Miraflores","Casa en venta ubicada en aurora miraflores, re...",https://www.adondevivir.com/propiedades/clasif...
9724,venta,4250000.0,1250000.0,280000.0,755522.058824,-112536.764706,Precio Muy Alto,quitar,"Francisco Quiñones Lastres\nMiraflores, Lima",Ubicada en una tranquila calle de una sola cua...,https://www.adondevivir.com/propiedades/clasif...
9726,venta,7371000.0,1950000.0,280000.0,755522.058824,-112536.764706,Precio Muy Alto,quitar,"Genaro Castro Iglesias\nMiraflores, Lima",(Cod. Pavt0264) Área terreno: 580m² // Área co...,https://www.adondevivir.com/propiedades/clasif...
...,...,...,...,...,...,...,...,...,...,...,...
42634,venta,3118603.0,854412.0,280000.0,755522.058824,-112536.764706,Precio Muy Alto,quitar,"Los Gladiolos 194\nMiraflores, Lima",Moderno edificio de 5 pisos con solo 9 exclusi...,https://urbania.pe/inmueble/clasificado/veclap...
42664,venta,3128334.0,804199.0,280000.0,755522.058824,-112536.764706,Precio Muy Alto,quitar,"Malecón Malta 1\nMiraflores, Lima","Vendo Lindo departamento con vista al mar, en ...",https://urbania.pe/inmueble/clasificado/veclap...
42665,venta,4370000.0,1150000.0,280000.0,755522.058824,-112536.764706,Precio Muy Alto,quitar,"Meliton Porras 16\nMiraflores, Lima","Vendo departamento en calle Meliton Porras 16,...",https://urbania.pe/inmueble/clasificado/veclap...
42675,venta,5017200.0,1356000.0,280000.0,755522.058824,-112536.764706,Precio Muy Alto,quitar,"Malecon Cisneros, Miraflores\nMiraflores, Lima","Vendo Departamento en Miraflores, Malecon Cisn...",https://urbania.pe/inmueble/clasificado/veclap...
